<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/Stock_NeurIPS2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='1.3'></a>
## 2.3. Import Packages

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import datetime
import torch
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as ts
# matplotlib.use('Agg')
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader


%matplotlib inline
#from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading_com import StockTradingEnv
from finrl.agents.stablebaselines3.models_add import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../mha-gru-drl")

import itertools

In [6]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [7]:
TRAIN_START_DATE = '2017-03-02'
TRAIN_END_DATE = '2020-02-27'
TRADE_START_DATE = '2020-02-28'
TRADE_END_DATE = '2023-12-01'

In [8]:
import tushare as ts
ts.set_token('df3c6a8e252ff736ca08a3364022e4340b68485bf9f9d3cea4c94f21')
pro = ts.pro_api()

In [9]:
#选择上证50指数的成分股
df_index = pro.index_weight(index_code='000016.sh', start_date='20121010', end_date='20220228')
# df_index.to_csv('SSE_50_index_weight.csv',index=0)

In [10]:
#选择某个时间点的上证50指数成分股作为参考跟踪股票池
select_date = '20160229' #自定义的时间点为每个月的月末
df_select = df_index[df_index['trade_date']==select_date]
sort_SSEindex = df_select['con_code'].unique()
print(sort_SSEindex,len(sort_SSEindex))

['601318.SH' '600016.SH' '601166.SH' '600000.SH' '600036.SH' '601328.SH'
 '601288.SH' '600030.SH' '600519.SH' '600837.SH' '601169.SH' '601398.SH'
 '601766.SH' '600887.SH' '601668.SH' '601601.SH' '601988.SH' '600104.SH'
 '600048.SH' '601818.SH' '601989.SH' '600015.SH' '600028.SH' '600637.SH'
 '601688.SH' '601390.SH' '600999.SH' '600518.SH' '601006.SH' '601857.SH'
 '600050.SH' '601186.SH' '601628.SH' '601985.SH' '600795.SH' '600585.SH'
 '600893.SH' '601088.SH' '600010.SH' '601901.SH' '601211.SH' '601669.SH'
 '600111.SH' '601336.SH' '600109.SH' '600958.SH' '601998.SH' '601800.SH'
 '600018.SH' '600150.SH'] 50


In [11]:
#随机选择成分股中的k只股票
import random 
k = 50
selected_tics = random.sample(list(sort_SSEindex),k)
print(selected_tics)

['600030.SH', '600109.SH', '601390.SH', '601166.SH', '600010.SH', '601169.SH', '600519.SH', '601288.SH', '601601.SH', '601628.SH', '601186.SH', '601328.SH', '601006.SH', '600795.SH', '601669.SH', '601901.SH', '600050.SH', '601985.SH', '601766.SH', '600048.SH', '600036.SH', '600518.SH', '601988.SH', '600887.SH', '600958.SH', '600016.SH', '600000.SH', '601336.SH', '600150.SH', '601668.SH', '601857.SH', '601318.SH', '601211.SH', '600111.SH', '600585.SH', '600999.SH', '600893.SH', '601688.SH', '600637.SH', '601800.SH', '600028.SH', '601818.SH', '601989.SH', '600018.SH', '600104.SH', '600015.SH', '601998.SH', '601398.SH', '601088.SH', '600837.SH']


In [12]:
addfeature_df = pd.read_csv('1227_addfeature_df.csv')

In [13]:
merged_df = pd.read_csv('1229_merged_df.csv')

In [14]:
merged_df['trade_date'] = merged_df['trade_date'].astype(str)

In [15]:
addfeature_df['date'] = pd.to_datetime(addfeature_df['date'])

In [16]:
merged_df1 = merged_df[['trade_date','ts_code','filled']]
merged_df1['date'] = pd.to_datetime(merged_df['trade_date'])
merged_df1['tic'] = merged_df1['ts_code']
merged_df1 = merged_df1.loc[:,('date','tic','filled')]

In [17]:
addfeature_df1 = pd.merge(addfeature_df,merged_df1,on=('date','tic'),how='left')
addfeature_df1

,tic,date,close,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,...,temporal_feature_11,temporal_feature_12,temporal_feature_13,temporal_feature_14,temporal_feature_15,temporal_feature_16,temporal_feature_17,temporal_feature_18,temporal_feature_19,filled
0,600000.SH,2017-03-17,16.20,-0.103361,16.920131,16.024869,42.451553,-109.290506,24.313216,16.560333,...,-0.444595,2.496382,1.498984,-1.503451,-0.268903,1.428747,0.559653,-1.066029,2.959662,0
1,600000.SH,2017-03-20,16.14,-0.110808,16.854846,16.013154,40.998417,-121.936983,25.144384,16.543000,...,-0.317013,2.691144,2.069022,-1.325521,0.301127,1.796261,-0.097198,-1.664815,3.305223,0
2,600000.SH,2017-03-21,16.00,-0.126338,16.798334,15.981666,37.869445,-145.491419,36.351888,16.520667,...,-0.027345,2.698100,2.387536,-1.106515,0.843225,1.773876,-0.257767,-1.632134,3.319397,0
3,600000.SH,2017-03-22,15.78,-0.154311,16.797444,15.885556,33.689702,-193.198499,50.992976,16.491000,...,0.032896,2.916779,2.445098,-1.038702,0.922410,1.836789,-0.038941,-1.385233,3.098996,0
4,600000.SH,2017-03-23,15.88,-0.166606,16.770306,15.831694,36.961361,-156.439114,50.992976,16.463000,...,-0.994125,2.443021,2.249985,-0.970342,-1.083996,1.311004,-1.812655,-2.678623,4.134655,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70545,601998.SH,2022-12-26,4.83,0.031323,5.190485,4.833515,50.959756,-36.727817,0.176333,4.928000,...,-0.934460,1.517315,-1.088990,-0.973530,-1.664009,0.232380,2.426849,-0.475797,-0.119264,0
70546,601998.SH,2022-12-27,4.90,0.025253,5.174477,4.825523,52.808796,-19.119878,2.625621,4.935333,...,-0.892610,1.788705,-0.985208,-1.021302,-1.598934,0.435053,2.181200,-0.728077,0.176656,0
70547,601998.SH,2022-12-28,4.98,0.026590,5.171599,4.823401,54.822659,8.610136,14.173296,4.946667,...,-0.915082,1.391061,-1.209658,-0.966595,-1.683521,0.205795,2.563425,-0.428341,-0.234574,0
70548,601998.SH,2022-12-29,4.93,0.023347,5.171576,4.818424,53.350646,-44.954584,9.544348,4.957333,...,-0.967534,0.878120,-1.595519,-0.918358,-1.892846,-0.190184,2.897044,-0.247745,-0.814026,0


<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [18]:
TRAIN_START_DATE = '2015-03-02'
TRAIN_END_DATE = '2020-07-30'
TRADE_START_DATE = '2021-12-29'
TRADE_END_DATE = '2022-04-29'

In [19]:
TRAIN_START_DATE = addfeature_df['date'].unique()[0]

In [ ]:
TRAIN_START_DATE

In [20]:
train = data_split(addfeature_df1, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(addfeature_df1, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

41050
3950


In [21]:
selfattn_indicator_list = [f"temporal_feature_{i}" for i in range(20)]

In [22]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 3*stock_dimension + len(INDICATORS)*stock_dimension + len(selfattn_indicator_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 50, State Space: 1551


In [28]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 500,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "risk_preference": 'dd',   #ne为中性、pr为激进,除此之外都是保守策略
    "reward_scaling": 1,
    "attn_indicator_list":selfattn_indicator_list,
    "alpha": 100, #风险喜好函数系数
    "beta" : 100, #风险中性函数系数
    "c" : 100, # 风险厌恶函数系数
    "theta" : 0.2, # 风险喜好放缩因子
    "gamma" : 5, # 风险厌恶放缩因子
    "mu" : 1, #惩罚项放缩因子
    "p_lambda" : 100,  #惩罚系数
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


## Environment for Training



In [29]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [30]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = True
if_using_sac = False


### Agent 4: TD3

In [31]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device
Logging to results/td3


In [32]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000) if if_using_td3 else None

pr: 0.0117856670684926 ne: 0.011716757000004872 dd: 0.011648383103350146 R: 0.05824191551675073
pr: 0.016320419756469473 ne: 0.016188673212486824 dd: 0.016058340892511236 R: 0.08029170446255618
pr: -0.24064808653141834 ne: -0.27528995493205705 dd: -0.31691246477826285 R: -1.5845623238913142
pr: 0.01543052455538918 ne: 0.015312684685962622 dd: 0.015196041661388326 R: 0.07598020830694163
pr: 0.2771161025328541 ne: 0.2446044911109016 dd: 0.21698583392947646 R: 1.0849291696473822
pr: -0.07083018649121631 ne: -0.0734637651516968 dd: -0.0762295389512746 R: -0.381147694756373
pr: -0.1315993538652972 ne: -0.1411020969263393 dd: -0.15154221090351894 R: -0.7577110545175947
pr: -0.14515737610062163 ne: -0.1568378926022932 dd: -0.16980596432883033 R: -0.8490298216441516
pr: -0.0997212615737878 ne: -0.10505085424541273 dd: -0.11076709614192559 R: -0.553835480709628
pr: 0.2761669116390233 ne: 0.24386098486244545 dd: 0.21640344152500635 R: 1.0820172076250318
pr: 0.9005273345717288 ne: 0.6421313921754

pr: -0.18194017477986002 ne: -0.20081980908775304 dd: -0.22240448579772254 R: -1.1120224289886127
pr: 0.45972186084188693 ne: 0.37824591129894 dd: 0.3149379845395648 R: 1.574689922697824
pr: -0.0913414490492459 ne: -0.09578588688917086 dd: -0.10052340227654577 R: -0.5026170113827289
pr: 6.812035213660677 ne: 2.0556655206413286 dd: 0.8719923844875495 R: 4.359961922437748
pr: 12.333611249730701 ne: 2.5902880089584017 dd: 0.9250015632471513 R: 4.625007816235756
pr: -0.7636215297802953 ne: -1.4423210711183465 dd: -3.2305037301854878 R: -16.15251865092744
pr: -0.600946000551456 ne: -0.9186585342857246 dd: -1.5059265196737983 R: -7.529632598368991
pr: 0.05272172719693735 ne: 0.05137893155767388 dd: 0.050081351828197285 R: 0.2504067591409864
pr: 0.4914093712836425 ne: 0.39972155965592004 dd: 0.3294932838330571 R: 1.6474664191652855
pr: 1.225767567378857 ne: 0.8001018302311761 dd: 0.5507167888255127 R: 2.7535839441275636
pr: 1.2046957198664616 ne: 0.7905895038511366 dd: 0.5464226691288856 R: 2

pr: -0.19635274478365905 ne: -0.21859484336964533 dd: -0.2443270271990179 R: -1.2216351359950894
pr: -0.5823294764831635 ne: -0.873062378518674 dd: -1.394231681900553 R: -6.971158409502765
pr: -0.9508463866087109 ne: -3.0128049174658353 dd: -19.344384288484832 R: -96.72192144242416
pr: 2.8657999888767587 ne: 1.3521686435131695 dd: 0.7413213298987673 R: 3.7066066494938363
pr: -0.553801629414467 ne: -0.8069916487484788 dd: -1.2411556516616802 R: -6.205778258308401
pr: 0.43430342794302534 ne: 0.36067931529148706 dd: 0.30279745518413215 R: 1.5139872759206607
pr: -0.6492506369997804 ne: -1.0476833761089288 dd: -1.8510386774369527 R: -9.255193387184764
pr: -0.6792586899302543 ne: -1.1371203683873188 dd: -2.1177773757379375 R: -10.588886878689687
pr: -0.9083043635762351 ne: -2.3892804861980115 dd: -9.90564435780314 R: -49.5282217890157
pr: -0.6314184299074803 ne: -0.9980932347454403 dd: -1.7131036414788308 R: -8.565518207394154
pr: 7.3101914743372 ne: 2.117482650038393 dd: 0.8796658292305163 

pr: 5.153125521984189 ne: 1.8169601676777218 dd: 0.8374809685862655 R: 4.187404842931327
pr: 1.0580038685065802 ne: 0.7217365171508978 dd: 0.5140922642066439 R: 2.5704613210332194
pr: 12.777639356749996 ne: 2.623046941530849 dd: 0.927418625636323 R: 4.637093128181615
pr: -0.17089523593643263 ne: -0.187408757810175 dd: -0.20612019535245385 R: -1.0306009767622692
pr: 10.17893926119196 ne: 2.414031584978139 dd: 0.9105460744856597 R: 4.552730372428298
pr: 0.14250677716785987 ne: 0.1332247756442178 dd: 0.12473166900691612 R: 0.6236583450345806
pr: -0.8588302191589079 ne: -1.9577919935339854 dd: -6.083668998010637 R: -30.418344990053185
pr: 12.080911725519554 ne: 2.5711540473857553 dd: 0.9235527292758119 R: 4.61776364637906
pr: -0.241952249356445 ne: -0.2770088997519308 dd: -0.3191781113406593 R: -1.5958905567032966
pr: -0.1353473841284455 ne: -0.14542745286168612 dd: -0.1565338283190385 R: -0.7826691415951925
pr: -0.37960337072773687 ne: -0.4773962808468934 dd: -0.6118720715375561 R: -3.059

pr: -0.26977219598460256 ne: -0.31439873322532186 dd: -0.369435667200251 R: -1.847178336001255
pr: -0.6050287572459597 ne: -0.9289423198856219 dd: -1.5318298948228195 R: -7.6591494741140975
pr: 18.862455912074463 ne: 2.9888313122039563 dd: 0.9496537586073586 R: 4.748268793036793
pr: -0.03552019381555105 ne: -0.03616638393749772 dd: -0.03682834372247923 R: -0.18414171861239614
pr: 0.6933087880490123 ne: 0.5266844775705775 dd: 0.40944025858852673 R: 2.0472012929426335
pr: -0.596515829004648 ne: -0.9076180212972539 dd: -1.4784119722295608 R: -7.3920598611478034
pr: 4.316439658156847 ne: 1.6708038421258207 dd: 0.8119041944799033 R: 4.0595209723995165
pr: -0.5461539589187979 ne: -0.7899972550150418 dd: -1.2033903779741908 R: -6.016951889870954
pr: -0.2249029808518891 ne: -0.2547670714663407 dd: -0.290161070544529 R: -1.450805352722645
pr: 0.3511416020094489 ne: 0.30094986622389475 dd: 0.25988512342986336 R: 1.2994256171493168
pr: -0.7979752335653789 ne: -1.5993649829844618 dd: -3.9498881629

...

pr: 4.144280026640982 ne: 1.6378854229810569 dd: 0.8056093379790288 R: 4.0280466898951435
pr: -0.16970075406425544 ne: -0.18596910589745677 dd: -0.204385051407584 R: -1.02192525703792
pr: -0.457623440445456 ne: -0.6117947594502979 dd: -0.8437374963647102 R: -4.218687481823551
pr: -0.7201700308369294 ne: -1.2735731132546941 dd: -2.573598649890324 R: -12.867993249451619
pr: -0.43000269941797664 ne: -0.5621236539857669 dd: -0.7543942734095435 R: -3.7719713670477173
pr: -0.4095396805403214 ne: -0.5268528438080761 dd: -0.6935939080801994 R: -3.467969540400997
pr: 12.486126344631064 ne: 2.601661478845627 dd: 0.9258497233048608 R: 4.629248616524304
pr: 10.944984530070904 ne: 2.480311485680317 dd: 0.916282855161298 R: 4.58141427580649
pr: 12.590445313931145 ne: 2.6093669953915537 dd: 0.9264188937962959 R: 4.63209446898148
pr: -0.37969412478760367 ne: -0.47754257548190976 dd: -0.6121078970235359 R: -3.0605394851176793
pr: -0.6404445505962226 ne: -1.0228868733705505 dd: -1.7812121931630345 R: -8

pr: -0.4437820082616508 ne: -0.5865949900996393 dd: -0.797856262927954 R: -3.98928131463977
pr: 2.1091944797787687 ne: 1.134363682924895 dd: 0.6783732871958676 R: 3.3918664359793382
pr: -0.24372527660611198 ne: -0.27935057808036895 dd: -0.3222708217886232 R: -1.611354108943116
pr: -0.4313661318822726 ne: -0.5645185174910442 dd: -0.7586008432986346 R: -3.7930042164931734
pr: 2.9561078641859453 ne: 1.375260679360936 dd: 0.747226305669557 R: 3.7361315283477845
pr: 0.5940346504831406 ne: 0.4662683182014793 dd: 0.3726610649919643 R: 1.8633053249598213
pr: -0.8072396302142639 ne: -1.6463074688380326 dd: -4.18778834628485 R: -20.93894173142425
pr: -0.06242539881398812 ne: -0.06445894970504891 dd: -0.06658179384874696 R: -0.3329089692437348
pr: 0.14255317373691123 ne: 0.13326538426723375 dd: 0.12476721172693184 R: 0.6238360586346592
pr: 1.0669642684007092 ne: 0.7260809938548635 dd: 0.5161987000511921 R: 2.5809935002559605
pr: 0.4624009435115384 ne: 0.3800795668977499 dd: 0.31619300135379735 R:

pr: -0.19874581740879205 ne: -0.22157705067825106 dd: -0.248043407107168 R: -1.24021703553584
pr: 0.24878451960805892 ne: 0.2221706939283333 dd: 0.19922133538790332 R: 0.9961066769395166
pr: -0.6161333828896065 ne: -0.9574601380098546 dd: -1.6050715415881478 R: -8.025357707940739
pr: -0.627803692605365 ne: -0.9883338558478241 dd: -1.6867542211796116 R: -8.433771105898058
pr: -0.8403378754764577 ne: -1.83469541830446 dd: -5.263226190833685 R: -26.316130954168425
pr: -0.8147645117855185 ne: -1.686127354098504 dd: -4.3985335620035935 R: -21.992667810017966
pr: -0.6344975854432842 ne: -1.0064823936397995 dd: -1.7359600379461457 R: -8.679800189730729
pr: -0.5148852544331985 ne: -0.723369827229841 dd: -1.06136797353297 R: -5.30683986766485
pr: 1.7128387671135354 ne: 0.9979956021700978 dd: 0.6313824425828294 R: 3.156912212914147
pr: -0.45308015671195456 ne: -0.6034530260767221 dd: -0.828421499552964 R: -4.14210749776482
pr: 1.28948553412348 ne: 0.8283271347388522 dd: 0.5632206514976537 R: 2.8

pr: 0.6676176355676897 ne: 0.511396042386965 dd: 0.40034215357792113 R: 2.001710767889606
pr: -0.4986818292057572 ne: -0.690514308024211 dd: -0.9947411808666167 R: -4.973705904333084
pr: -0.7533011639865029 ne: -1.3995869736845599 dd: -3.0535254083861565 R: -15.267627041930783
pr: 1.410843610914061 ne: 0.8799767322731803 dd: 0.5852074371506601 R: 2.9260371857533007
pr: 1.577512968092254 ne: 0.9468249681639269 dd: 0.6120291100843035 R: 3.0601455504215176
pr: -0.9546460550612308 ne: -3.0932581174772964 dd: -21.04879865136462 R: -105.2439932568231
pr: 221.11729231416933 ne: 5.403205586105378 dd: 0.9954978741655757 R: 4.977489370827878
pr: 0.42317358303991126 ne: 0.3528892955333 dd: 0.2973450238838812 R: 1.4867251194194058
pr: 1.8645495608928915 ne: 1.0524111164773005 dd: 0.6509049752002559 R: 3.2545248760012795
pr: 0.9094864845854611 ne: 0.6468343496722424 dd: 0.4762989902926208 R: 2.381494951463104
pr: -0.8940568114132504 ne: -2.2448522852749933 dd: -8.439021171060645 R: -42.195105855303

KeyboardInterrupt: 

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
e_trade_gym = StockTradingEnv(df = trade,**env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trained_moedl = trained_td3
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
#now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
#perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
print("==============Get Backtest Results===========")
#now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
#perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
print("==============Get Backtest Results===========")
#now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
#perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#获取随机选择的tic在select_date的权重并修正
def get_selected_index_weight(df_index,select_date,selected_tics):
    df = df_index[df_index['trade_date'] == select_date]
    select_df = df[df['con_code'].isin(selected_tics)]
    select_df = select_df.drop('index_code',axis=1).rename(columns={'con_code':'tic','trade_date':'date'})
    select_df = select_df.reset_index()
    weight_sum = select_df['weight'].sum()
#     print(select_df)
    for i in range(len(select_df)):
        select_df.loc[i,'weight'] = (select_df.loc[i,'weight'] / weight_sum)*100
    return select_df.loc[:,['tic','date','weight']]

baseline_weight =get_selected_index_weight(df_index,select_date,selected_tics)
baseline_weight = baseline_weight.sort_values(['tic'],ascending=True).reset_index(drop=True)
print(baseline_weight)

In [ ]:
#计算所选股票的buy & hold策略
def calculate_selected_baseline(df,full_date_range,selected_tics):
    df = df.sort_values(['tic','date'],ascending=True).reset_index(drop=True)
    baseline = pd.DataFrame({'date':full_date_range['date']})
    for i in range(len(full_date_range)):
        temp_date = full_date_range.loc[i,'date']
        temp_df = df[df['date'] == temp_date].sort_values('tic',ascending=True)
        close = list(temp_df['close'])
        weight = list(baseline_weight['weight'])
        baseline.loc[i,'account_value'] = sum(np.array(close) * np.array(weight)) #注意权重和收盘价对应的股票顺序
    baseline['date'] = pd.to_datetime(baseline['date'],format='%Y%m%d')
#     baseline.set_index("date", inplace=True, drop=True)
#     baseline.index = baseline.index.tz_localize("UTC")
    return baseline
# full_date_range = get_trading_days(exchange='SSE',start_date='20210301', end_date='20230227')
# full_date_range = full_date_range.sort_values('trade_date',ascending=True).reset_index(drop=True)
# subset_df = addfeature_df.loc[(addfeature_df['date'].astype(str) >='20210301') & (addfeature_df['date'].astype(str)<='20220227')]
full_date_range1 = pd.DataFrame({'date':df_account_value['date'].unique()}).reset_index(drop=True)
baseline_sse = calculate_selected_baseline(addfeature_df,full_date_range1,selected_tics)
baseline_sse

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
# baseline_df = get_baseline(
#         ticker="^DJI", 
#         start = df_account_value.loc[0,'date'],
#         end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_sse, value_col_name = 'account_value')


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
import pyfolio
from copy import deepcopy
def backtest_plot_com(
    account_value,
    baseline,
    baseline_start=TRADE_START_DATE,
    baseline_end=TRADE_END_DATE,
    value_col_name="account_value",
):
    df = deepcopy(account_value)
    df["date"] = pd.to_datetime(df["date"])
    test_returns = get_daily_return(df, value_col_name=value_col_name)
#     pro = ts.pro_api()
#     baseline_df = pro.index_daily(ts_code='000016.sh',start_date = '20200301',end_date='20230227')
#     baseline_df = baseline_df.rename(columns={'trade_date':'date'})
#     baseline_df = baseline_df.sort_values('date',ascending=True)
    baseline_returns = get_daily_return(baseline, value_col_name=value_col_name)
    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(
            returns=test_returns, benchmark_rets=baseline_returns, set_context=False
        )

In [ ]:
print("==============Compare to SSE50===========")
# from finrl.plot import backtest_plot_com
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot_com(account_value = df_account_value, 
                  baseline = baseline_sse,
                  baseline_start = df_account_value.loc[len(df_account_value)-1,'date'],
                  baseline_end = df_account_value.loc[0,'date']
                     )

In [ ]:
print("==============Compare to SSE50===========")
# from finrl.plot import backtest_plot_com
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot_com(account_value = df_account_value, 
                  baseline = baseline_sse,
                  baseline_start = df_account_value.loc[len(df_account_value)-1,'date'],
                  baseline_end = df_account_value.loc[0,'date']
                     )

In [ ]:
print("==============Compare to SSE50===========")
# from finrl.plot import backtest_plot_com
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot_com(account_value = df_account_value, 
                  baseline = baseline_sse,
                  baseline_start = df_account_value.loc[len(df_account_value)-1,'date'],
                  baseline_end = df_account_value.loc[0,'date']
                     )

In [ ]:
print(df_account_value.loc[len(df_account_value)-1,'date'],df_account_value.loc[0,'date'])